In [1]:
import pandas as pd
import numpy as np
import pickle
import string

In [2]:
# 获取初始训练数据
train_file = 'train.csv'
train = pd.read_csv(train_file)
train.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [4]:
# 获取训练集标签
labels = pd.read_csv(train_file, usecols=(i for i in range(2, 8)))
labels.head(3)

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0


In [5]:
# 将标签转成numpy.array类型
label_array = np.array(labels)

In [6]:
# 将获取评论列
comments = pd.read_csv(train_file, usecols=(1,))
comments.head(3)

,comment_text
0,Explanation\nWhy the edits made under my usern...
1,D'aww! He matches this background colour I'm s...
2,"Hey man, I'm really not trying to edit war. It..."


In [10]:
# 获取评论到一个list中去
comments_list = []
for i in comments.comment_text:
    comments_list.append(i)

print('The number of the comments:', len(comments_list))

The number of the comments: 159571


In [11]:
# 为全0的添加一个label
new_label = np.zeros((label_array.shape[0], label_array.shape[1]+1))
new_label

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [12]:
# 添加最后一个label，用来标记是否全为0
for i, raw in enumerate(label_array):
    if (raw == np.array([0, 0, 0, 0, 0, 0])).all():
        temp = np.append(raw, 1)
        new_label[i] = temp
    else:
        temp = np.append(raw, 0)
        new_label[i] = temp

In [13]:
comments_list_new = comments_list

for i, comments in enumerate(comments_list):
    # 删除句子中的标点符号
    for j in string.punctuation:
        comments_list_new[i] = comments_list_new[i].replace(j, '')
    # 删除换行符号
    comments_list_new[i] = comments_list_new[i].replace('\n', ' ')
    comments_list_new[i] = comments_list_new[i].lower()


In [14]:
comments_list_new[0]

'explanation why the edits made under my username hardcore metallica fan were reverted they werent vandalisms just closure on some gas after i voted at new york dolls fac and please dont remove the template from the talk page since im retired now892053827'

In [15]:
comments_splits = []

# 将单词分成list
for sentence in comments_list_new:
    temp = sentence.split(' ')
    while '' in temp:
        temp.remove('')
    comments_splits.append(temp)
    
comments_splits[0][0:5]

['explanation', 'why', 'the', 'edits', 'made']

In [16]:
print('The number of the comments is:', len(comments_splits),
      '\nThe number of the labels is:', labels.shape[0]) 

The number of the comments is: 159571 
The number of the labels is: 159571


In [17]:
# 设定长度统一为50个词
max_len = 50
sentence_indx = []
for sentence in comments_splits:
    if len(sentence) < max_len:
        pre = ['<pad>' for i in range(max_len - len(sentence))]
        pre.extend(sentence)
        sentence_indx.append(pre)
    else:
        pre = sentence[:max_len]
        sentence_indx.append(pre)

In [18]:
print(sentence_indx[0])

['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'explanation', 'why', 'the', 'edits', 'made', 'under', 'my', 'username', 'hardcore', 'metallica', 'fan', 'were', 'reverted', 'they', 'werent', 'vandalisms', 'just', 'closure', 'on', 'some', 'gas', 'after', 'i', 'voted', 'at', 'new', 'york', 'dolls', 'fac', 'and', 'please', 'dont', 'remove', 'the', 'template', 'from', 'the', 'talk', 'page', 'since', 'im', 'retired', 'now892053827']


In [19]:
# 分别保存
with open('processed_data/train_split_list.pkl', 'wb') as f:
    pickle.dump(sentence_indx, f)
    
with open('processed_data/labels_array.pkl', 'wb') as f:
    pickle.dump(new_label, f)

In [23]:
# 字符转换成索引
word2ind = pickle.load(open('processed_data/word2ind.pkl', 'rb'))

In [28]:
index_word = []

for sentence in sentence_indx:
    temp = []
    for word in sentence:
        if word in word2ind.keys():
            temp.append(word2ind[word])
        else:
            temp.append(word2ind['<OOV>'])
            
    index_word.append(temp)

In [30]:
index_word[2000]

[4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 968,
 6,
 968,
 33,
 6,
 722,
 120,
 774,
 7,
 63,
 1]

In [31]:
with open('processed_data/train_index_list.pkl', 'wb') as f:
    pickle.dump(index_word, f)

In [32]:
ls processed_data/

ind2word.pkl          test_split_list.pkl   train_split_list.pkl
labels_array.pkl      train_index_list.pkl  word2ind.pkl
